# Chapter 2. Basics of NumPy

This chapter introduces the basics of [NumPy](https://numpy.org/), the standard open-source library for efficient numerical computing in Python.
NumPy provides typed multidimensional arrays along with a comprehensive collection of numerical routines, forming the foundation of many scientific Python libraries such as SciPy and Matplotlib.

## Introduction

This section summarizes the advantages of using NumPy and outlines the chapter structure.
It includes technical background; if you prefer to proceed directly to the practical exercises, you may skip these details.

### Why NumPy?

NumPy addresses performance limitations of Python that make many scientific computations impractical in pure Python.

Python is an interpreted language, which enables for rapid development and flexibility.
However, compared with compiled languages such as C or Java (whose code is translated to machine code in advance), Python is substantially slower for many numerical workloads.
For example, [a benchmark enumerating prime numbers](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#primes) shows Python can be more than 30× slower than optimized C++ implementations (see the linked benchmark).

Why is Python relatively slow?
Several factors contribute: it can't exploit compiler optimizations the way compiled languages can, and its dynamic typing requires runtime type inspection and method dispatch.
For instance, even a simple expression like `a + b` typically involves:

1. Checking whether a implements the operation `+`.
2. Looking up the function that performs addition for `a`.
3. Verifying that `b` is compatible with that operation.
4. Extracting the values and computing the sum.

These runtime checks and dispatch steps add measurable overhead.
By contrast, statically typed languages resolve such operations at compile time, avoiding much of this cost and enabling faster execution.

NumPy allows Python programs to take advantage of the performance benefits of statically typed, compiled code.
Its computational kernels are implemented in C (with parts written in C++ and Fortran), and it relies on optimized linear-algebra libraries such as BLAS and LAPACK. As a result, operations such as matrix multiplication achieve high performance.

### Why not use a statically typed language?

One might wonder why numerical code is not simply written directly in C or another statically typed language.
For most practitioners, especially beginners, that is not the most productive choice.
Libraries such as NumPy already incorporate extensive optimizations and domain expertise.
Reimplementing these features without comparable expertise is rarely efficient.
For example, a naive C implementation of [matrix multiplication](https://github.com/kostya/benchmarks/tree/master?tab=readme-ov-file#matmul) (three nested loops) can be orders of magnitude slower than a tuned library call such as NumPy's `np.dot`, due to missing algorithmic and low-level optimizations.
Consequently, researchers and engineers typically focus on higher-level problems rather than low-level optimization tasks (see the [*reinventing the wheel*](https://en.wikipedia.org/wiki/Reinventing_the_wheel#Related_phrases) article).
In recent years, the AI and machine-learning communities have standardized on delegating low-level numerical computation to optimized libraries while using Python's flexibility for rapid prototyping and experimentation.

### What this chapter covers

This chapter begins with how to construct and manipulate multidimensional arrays (matrices and tensors).
It then introduces array indexing, logical operations, elementwise selection, and search techniques such as `a[idx]`.
We also review commonly used mathematical and statistical routines, followed by essential linear-algebra tools that appear throughout this RC bootcamp.

### Tips for using NumPy

Lastly, here are two tips for coding with NumPy before we move on to the exercise:
1. Avoid using `for` loops whenever possible.
2. Be mindful of array element types.

First, try to avoid `for` loops.
Each Python-level operation incurs overhead due to dynamic type checking.
Therefore, using `for` loops slows down the process proportionally to the number of iterations.
Instead, strive to complete operations on NumPy arrays whenever possible.
In fact, most problems in the exercise section can be solved without resorting to `for` loops.

Next, pay attention to the types of array elements.
NumPy achieves high performance by ensuring that all elements share the same fixed data type.
Using unsupported or mixed types can significantly degrade performance.
By keeping these points in mind, you can perform efficient numerical computations.

### Preparation

As in the previous chapter, begin by running the initialization cell below.

In [ ]:
import sys

import numpy as np

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/rc-bootcamp/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/rc-bootcamp/
    else:
        pass
        %cd /content/
        !git clone --branch en https://github.com/rc-bootcamp/rc-bootcamp.git
        %cd /content/rc-bootcamp/
else:
    sys.path.append(".")

from utils.tester import load_from_chapter_name

test_func, show_solution = load_from_chapter_name("02_numpy_basics")

## 1. Constructing arrays

We begin with array creation.
Use `np.array` to construct arrays from Python sequences; for filled arrays, use `np.zeros` or `np.ones`.
Experiment interactively in a blank cell before attempting the exercises.

Q1.1.

A list of integers $A$ is given.
The list $A$ may contain very large integers.
If you convert it directly to an `np.ndarray` type using the `np.array` function, the resulting array may have the `object` dtype, as shown below:
```py
In [*]: np.array([10**20])
Out[*]: array([100000000000000000000], dtype=object)
```
The `object` type in `numpy` only holds pointers (variables that store memory addresses), and computations are performed on Python itself (*not* `numpy`), which takes considerably more computation time.
For example:
```py
In [*]: x_int = np.arange(100000, dtype=np.int64)
   ...: x_obj = np.arange(100000, dtype='object')
   ...: %timeit x_int * x_int
   ...: %timeit x_obj * x_obj
54.3 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.45 ms ± 9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
```
Therefore, we want to convert the elements to a data type supported by `numpy`, but casting with `np.int64` can lead to overflow if the values exceed the maximum value of a 64-bit signed integer ($2^{63} - 1$ or `9223372036854775807`, as obtained with `np.iinfo(np.int64).max`).
```py
In [*]: np.array([10**20], dtype=np.int64)
---------------------------------------------------------------------------
OverflowError                             Traceback (most recent call last)
Cell In[*], line 1
----> 1 np.array([10**20], dtype=np.int64)

OverflowError: Python int too large to convert to C long
```
To prevent this, we want to change the data type according to the range of integers in the list $A$.
If the minimum and maximum elements of $A$, denoted as $a_\text{min}$ and $a_\text{max}$ respectively, satisfy the conditions $-2^{63} \leq a_\text{min} \leq a_\text{max} \leq 2^{63}-1$, then $A$ should be converted to an `np.ndarray` with elements of type `np.int64`.
Otherwise, it should be converted to an `np.ndarray` with elements of type `np.float64`.
Write code that achieves such process.

- $A$: `list` of `int`
- $1 \leq |A| \leq 10^{4}$
- $0 \leq |a_\text{min}|, |a_\text{max}| \leq 10^{100}$
- Return: `np.ndarray`
- Sample
  - `[2**100, 10]`->`array([1.2676506e+30, 1.0000000e+01])`
  - `[2**60, 10]`->`array([1152921504606846976, 10])`
  - `[-2**63, 2**63 - 1]`->`array([-9223372036854775808,  9223372036854775807])`
  - `[-2**63, 2**63]`->`array([-9.22337204e+18,  9.22337204e+18])`

<details><summary>tips</summary>

- [`np.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html)
- [`np.asarray`](https://numpy.org/doc/stable/reference/generated/numpy.asarray.html)
- [`np.dtype`](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html)
- [`np.iinfo`](https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "01_01")
# show_solution("01_01")  # Uncomment it to see the solution.

Q1.2.

Some integers $a, d, n$ are given.
Write code to output an `np.ndarray` of length $n$ containing the arithmetic sequence $a_0,~\ldots,~a_{n-1}$, following the rule $a_{0}=a, a_{k+1}=a_{k}+d$.

- $a, d, n$: `int`
- $0 \leq |a|, |d| \leq 10^{4}$
- $1 \leq n \leq 10^{4}$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int64`
- Sample
  - `5, 3, 4`->`array([5, 8, 11, 14])`
  - `4, -2, 3`->`array([4, 2, 0])`

<details><summary>tips</summary>

- [`np.arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)

</details>

In [ ]:
def solution(a, d, n):
    # TODO
    ...


test_func(solution, "01_02")
# show_solution("01_02")  # Uncomment it to see the solution.

Q1.3.

Given non-negative real numbers $a, b$ and an integer $n$, write code to output an `np.ndarray` of length $n$ containing the geometric sequence $a_0,~\ldots,~a_{n-1}$, satisfying $a_{0}=a, a_{n-1}=b$.

- $a, b$: `float`
- $n$: `int`
- $0 \leq a, b \leq 10^{4}$
- $2 \leq n \leq 100$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- Sample
  - `1.0, 8.0, 4`->`array([1., 2., 4., 8.])`
  - `4.0, 1.0, 3`->`array([4., 2., 1.])`
  - `2.0, 2.0, 3`->`array([2., 2., 2.])`

<details><summary>tips</summary>

- [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html)
- [`np.logspace`](https://numpy.org/doc/stable/reference/generated/numpy.logspace.html)

</details>

In [ ]:
def solution(a, b, n):
    # TODO
    ...


test_func(solution, "01_03")
# show_solution("01_03")  # Uncomment it to see the solution.

Q1.4.

A matrix $A\in \mathbb{R}^{m\times n}$ is given.
Write code that creates a matrix $B=(b_{ij})_{0\leq i < m, 0\leq j < n}$ where each element $b_{ij}$ equals the element at the 0th row and 0th column of $A$ denoted as $a_{00}$, i.e. $b_{ij}=a_{00}$.
Ensure that the data type of $B$ matches that of $A$.

- $A$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64` | `np.float64`
- $1 \leq |m|, |n| \leq 10^{3}$
- $0 \leq |a_{ij}| \leq 10^{4}$
- Return: `np.ndarray`
  - `shape`: `(M, N)`
  - `dtype`: `np.int64` | `np.float64`
- Sample
  - `array([[2., 0., 0.]])`->`array([[2., 2., 2.]])`
  - `array([[4, 3, 2], [2, 3, 4]])`->`array([[4, 4, 4], [4, 4, 4]])`

<details><summary>tips</summary>

- [`np.full_like`](https://numpy.org/doc/stable/reference/generated/numpy.full_like.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "01_04")
# show_solution("01_04")  # Uncomment it to see the solution.

Q1.5.

Integers $m,n$ are given.
Write code to create a matrix $A=(a_{ij})_{0 \leq i < M, 0 \leq j < N}$ with a "3-colored checkerboard" pattern, that satisfies $a_{ij}= k~(i + j \equiv k~(\text{mod} 3), 0 \leq k \leq 2)$ .

- $m, n$: `int`
- $1 \leq m, n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int8`
- Sample
  - `3, 3`->`array([[2, 0, 1],[0, 1, 2], [1, 2, 0]], dtype=int8)`
  - `1, 10`->`array([[2, 0, 1, 2, 0, 1, 2, 0, 1, 2]], dtype=int8)`
  - `5, 1`->`array([[2], [0], [1], [2], [0]], dtype=int8)`

<details><summary>tips</summary>

- [Indexing on ndarrays](https://numpy.org/doc/stable/user/basics.indexing.html)

</details>

In [ ]:
def solution(m, n):
    # TODO
    ...


test_func(solution, "01_05")
# show_solution("01_05")  # Uncomment it to see the solution.

Q1.6.

A list $A$ of length $n$ containing single-digit integers is given.
For an integer $k~(0\leq k \leq 9)$, the *one-hot vector* $e^k \in \{0, 1\}^{10}$ is defined as a vector of length `10` where the $k + 1$-th element is `1` and all other elements are `0`.
Write code that constructs a matrix from $A$, where each integer $a_{i}$ (the $i$-th element of $A$) is replaced with its one-hot vector $e^{a_{i}}$.
In other words, the resulting matrix is $[e^{a_{1}}, e^{a_{2}},~\ldots,~e^{a_{n}}]^\top $.

- $A$: `list` of `int`
- $0 \leq n (=|A|) \leq 10^{3}$
- $0 \leq a_{i} \leq 9$
- Return: `np.ndarray`
  - `shape`: `(..., 10)`
  - `dtype`: `np.int8`
- Sample
  - `[2, 3]`->`array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=int8)`

<details><summary>tips</summary>

- [`np.eye`](https://numpy.org/doc/stable/reference/generated/numpy.eye.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "01_06")
# show_solution("01_06")  # Uncomment it to see the solution.

Q1.7.

2 square diagonal matrices $A, B \in \mathbb{R}^{n\times n}$ are given (a diagonal matrix is defined as a matrix with nonzero elements only at the $(i, i)$ positions).
Under this condition, the matrix product $A B$ coincides with the Hadamard product $A\odot B$, but the numpy code `a * b` for this operation has a time complexity of $O(n^2)$ and is not efficient for large values of $n$.
Write code that extracts the diagonal elements $a_{ii}, b_{ii}$ of $A, B$, computes the product $a_{ii}b_{ii}$ for $0\leq i < n$, and then reconstructs the square diagonal matrix $C=(a_{ij} b_{ij})$.

- $A, B$: `np.ndarray`
  - `dtype`: `np.int64`
  - `shape`: `(n, n)`
- $0 \leq |a_{ii}, b_{ii}| \leq 10^{3}$
- Return: `np.int64`
- Sample
  - `array([[1, 0], [0, 2]]), array([[3, 0], [0, 4]])`->`array([[3, 0], [0, 8]])`

<details><summary>tips</summary>

- [`np.multiply`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html)
- [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)

</details>

In [ ]:
def solution(mat_a, mat_b):
    # TODO
    ...


test_func(solution, "01_07")
# show_solution("01_07")  # Uncomment it to see the solution.

Q1.8.

A list $A$ containing $n$ integers and a list $B$ containing $n-1$ integers are given.
The $i$-th elements of $A$ and $B$ are denoted as $a_{i}$ and $b_{i}$, respectively.
Write code to create a tridiagonal matrix $D(\{a_i\},\{b_i\})$ represented by the following formula:

$$
\begin{align*}
D(\{a_i\},\{b_i\}) &= \begin{bmatrix}
a_{1} & b_{1} &        &         & \huge{0} \\
b_{1} & a_{2} & b_{2}  & \\
      & b_{2} & a_{3}  & \ddots  & \\
      &       & \ddots & \ddots  & b_{n-1} \\
\huge{0} &    &        & b_{n-1} & a_{n} \\
\end{bmatrix}
.\end{align*}
$$

- $A, B$: `list` of `int`
- $1 \leq |B| < |B| + 1 = |A| \leq 10^{3}$
- $0 \leq |a_i|, |b_i| \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, m)`
  - `dtype`: `np.int64`
- Sample
  - `[1, 2, 3], [4, 5]`->`array([[1, 4, 0], [4, 2, 5], [0, 5, 3]])`

<details><summary>tips</summary>

- [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)

</details>

In [ ]:
def solution(arr_a, arr_b):
    # TODO
    ...


test_func(solution, "01_08")
# show_solution("01_08")  # Uncomment it to see the solution.

Q1.9.

A matrix $C(\{a_{i}\})$ defined in the following manner for a sequence $a_{i}$ ($1\leq i \leq n$) is called a circulant matrix:

$$
\begin{align*}
C(\{a_{i}\})=\begin{bmatrix}
a_1     & a_{n}   & \cdots & a_{3}  & a_{2}   \\
a_2     & a_{1}   & a_{n}    &        & a_{3}   \\
\vdots  & a_{2}   & a_{1}    & \ddots & \vdots  \\
a_{n-1} &         & \ddots   & \ddots & a_{n}   \\
a_{n}   & a_{n-1} & \cdots & a_{2}  & a_{1}   \\
\end{bmatrix}
.\end{align*}
$$

Now, 2 integers $k, n$ are given.
Create code to generate the circulant matrix $C(\{b_{i}\})$ for the sequence $\{b_i\}$ with length $n$ defined by

$$
\begin{align*}
b_{i}=\begin{cases}
1&(i=k)\\
0&(\text{otherwise})
\end{cases}
.\end{align*}
$$

- $k, n$: `int`
- $1 \leq k \leq n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, n)`
  - `dtype`: `np.int8`
- Sample
  - `2, 3`->`array([[0, 0, 1], [1, 0, 0], [0, 1, 0]], dtype=np.int8)`

<details><summary>tips</summary>

- [`np.roll`](https://numpy.org/doc/stable/reference/generated/numpy.roll.html)

</details>

In [ ]:
def solution(k, n):
    # TODO
    ...


test_func(solution, "01_09")
# show_solution("01_09")  # Uncomment it to see the solution.

Q1.10.

Given a nonzero integer $n~(n\neq 0)$, write code to create a triangular matrix $T(n)$ according to the sign of $n$ as follows:

$$
\begin{align*}
T(n)=\begin{cases}
&
\begin{bmatrix}
1 & 2 & \cdots & n-1 & n \\
  & n + 1 & \cdots & 2n-2 & 2n-1 \\
  &       & \ddots & \vdots & \vdots \\
  & \huge{0} &     & m-2  & m-1 \\
  &       &        &      & m \\
\end{bmatrix}
& (n > 0)\\
\\
&
\begin{bmatrix}
1      &        & \\
2      & 3      & \huge{0} \\
\vdots & \vdots & \ddots \\
m-|n|+1  & m-|n|+2  & \cdots &  m \\
\end{bmatrix}
& (n < 0)\\
\end{cases}
,\end{align*}
$$

where $m=|n|(|n|+1)/2$.

- $n$: `int`
- $1 \leq |n| \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, n)`
  - `dtype`: `np.int32`
- Sample
  - `2`->`array([[1, 2], [0, 3]], dtype=int32)`
  - `-3`->`array([[1, 0, 0], [2, 3, 0], [4, 5, 6]], dtype=int32)`

<details><summary>tips</summary>

- [`np.triu`](https://numpy.org/doc/stable/reference/generated/numpy.triu.html)
- [`np.tril`](https://numpy.org/doc/stable/reference/generated/numpy.tril.html)
- [`np.triu_indices_from`](https://numpy.org/doc/stable/reference/generated/numpy.triu_indices_from.html)
- [`np.tril_indices_from`](https://numpy.org/doc/stable/reference/generated/numpy.tril_indices_from.html)

</details>

In [ ]:
def solution(n):
    # TODO
    ...


test_func(solution, "01_10")
# show_solution("01_10")  # Uncomment it to see the solution.

## 2. Array manipulation

Next we examine array reshaping and dimension manipulation.
Use `np.reshape` to change the shape of an array, for example:

```py
arr = np.arange(24)
arr = arr.reshape(4, 6)
```

This converts a 1-D array into a 2-D matrix.
The section covers related techniques for inserting, removing, and permuting axes.

Q2.1.

A tensor $A$ containing information for $n$ RGBA images, each of height $h$ and width $w$ pixels, is given.
Specifically, $a_{ijk}$ represents the RGBA value at the $(j, k)$ coordinate of the $i$-th image, stored as a 4-dimensional vector in the order of `Red`, `Green`, `Blue`, `Alpha`.
Now, we want to keep only the RGB values, omitting the Alpha channel, and represent each image's information as a vector of length $p(=h \times w \times 3)$.
Write code to transform the tensor $A$ into a matrix (2-dimensional tensor) $B\in \mathbb{R}^{n\times p}$.

- $A$: `np.ndarray`
  - `shape`: `(n, h, w, 4)`
  - `dtype`: `np.uint8`
- $1 \leq n \leq 10^{2}$
- $1 \leq w, h \leq 10^{2}$
- $0 \leq a_{ijkl} \leq 255$
- Return: `np.ndarray`
  - `shape`: `(n, h * w * 3)`
  - `dtype`: `np.uint8`

<details><summary>tips</summary>

- [`np.shape`](https://numpy.org/doc/stable/reference/generated/numpy.shape.html)
- [`np.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "02_01")
# show_solution("02_01")  # Uncomment it to see the solution.

Q2.2.

Consider a 4-dimensional tensor $A\in \mathbb{R}^{t\times w \times h \times d}$ that stores sensor values (16-bit integers) of XYZ coordinates for each point on a $w \times h \times d$ grid, over $t$ time steps.
To use this data in the ZXY coordinate system, we want to construct a 4-dimensional tensor $B$ such that $B_{ijkl}=A_{iljk}$.
Write code to perform this transformation.

- $A$: `np.ndarray`
  - `shape`: `(t, w, h, d)`
  - `dtype`: `np.uint16`
- $1 \leq t, w, h, d \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(t, d, w, h)`
  - `dtype`: `np.uint16`

<details><summary>tips</summary>

- [`np.moveaxis`](https://numpy.org/doc/stable/reference/generated/numpy.moveaxis.html)
- [`np.swapaxes`](https://numpy.org/doc/stable/reference/generated/numpy.swapaxes.html)
- [`np.transpose`](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "02_02")
# show_solution("02_02")  # Uncomment it to see the solution.

Q2.3.

A positive integer $k(\geq 2)$ and a matrix $A\in\mathbb{R}^{t \times p}$ that records $p$-dimensional vectors over $t$ time steps are given.
Here, $A$ is a 2-dimensional tensor.
We want to transform $A$ into a $k$-dimensional tensor $B\in\mathbb{R}^{t\times \underbrace{1 \times \cdots \times 1}_{k-2}\times p}$ such that $b_{i,\underbrace{0,~\ldots,~0}_{k-2},j}=a_{ij}$.
Write code to perform this transformation.

- $A$: `np.ndarray`
  - `shape`: `(t, p)`
  - `dtype`: `np.int64`
- $2 \leq k \leq 10$
- $1 \leq t, p \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(t, ... ,p)`
  - `dtype`: `np.int64`
- Sample
  - `3, array([[1, 2], [3, 4]])`->`array([[[1, 2]], [[3, 4]]])`

<details><summary>tips</summary>

- [`np.expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html)

</details>

In [ ]:
def solution(k, arr):
    # TODO
    ...


test_func(solution, "02_03")
# show_solution("02_03")  # Uncomment it to see the solution.

Q2.4.

Some positive integers $k, m, n$ are given.
Consider a square matrix $S\in\mathbb{R}^{k\times k}$ arranged with numbers from $1$ to $k^2$ as follows:

$$
\begin{align*}
\begin{bmatrix}
1 & 2 & \cdots & k-1 & k \\
k+1 & k+2 & \cdots & 2k-1 & 2k \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
k^2-k+1  & k^2-k+2 & \cdots & k^2-1 & k^2 \\
\end{bmatrix}
.\end{align*}
$$

Write code to output the block matrix $T \in \mathbb{R}^{km\times kn}$ formed by arranging $S$ $m$ times in the column direction and $n$ times in the row direction.

- $k, m, n$: `int`
- $1 \leq k, n, m \leq 10$
- Return: `np.ndarray`
  - `shape`: `(k * m, k * n)`
  - `dtype`: `np.int64`
- Sample
```py
In [*]: print(str(solution(2, 3, 4)))
[[1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]
 [1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]
 [1 2 1 2 1 2 1 2]
 [3 4 3 4 3 4 3 4]]
```

<details><summary>tips</summary>

- [`np.tile`](https://numpy.org/doc/stable/reference/generated/numpy.tile.html)
- [`np.repeat`](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html)

</details>

In [ ]:
def solution(k, m, n):
    # TODO
    ...


test_func(solution, "02_04")
# show_solution("02_04")  # Uncomment it to see the solution.

Q2.5.

A 3-dimensional tensor $A\in\mathbb{R}^{(t+1)\times n \times 3}$ is given, which records the XYZ coordinates of $n$ drones over time $k \in [0, t]$.
Now, we can obtain the velocity vector by computing the difference between the coordinates at time $k+1$ and $k$, and this data is arranged in a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$.
Write code to construct a 3-dimensional tensor $C\in\mathbb{R}^{t\times n \times 6}$, which combines the position and velocity into a 6-dimensional vector for each time $k\in[1, t]$.

- $A$: `np.ndarray`
  - `shape`: `(t + 1, n, 3)`
  - `dtype`: `np.int64`
- $1 \leq t \leq 10^{3}$
- $1 \leq n \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t, n, 6)`
  - `dtype`: `np.int64`

<details><summary>tips</summary>

- [`np.concatenate`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "02_05")
# show_solution("02_05")  # Uncomment it to see the solution.

## 3. Logical operations and search

Next, we will cover logical operations and search methods.
In NumPy, you can execute the following code involving logical operations and indexing:

```py
arr = np.arange(10)
arr[arr >= 5] = 5
```

Executing this code assigns the value 5 to all elements greater than or equal to 5.
In this way, NumPy allows you to intuitively write code for searching, assigning, and manipulating array elements.
Furthermore, let's learn typical methods for searching values using functions such as `np.argsort`.

Q3.1.

Some positive integers $h, w, x, y, r$ are given.
Now, the task of drawing a circle on a display with height $h$ px and width $w$ px is represented by the matrix $A=(a_{ij})_{0\leq i < h, 0\leq j < w}$.
Each $a_{ij}$ corresponds to the coordinate $(i, j)$ on the XY plane.
Write code to create a matrix $A\in\mathbb{R}^{h \times w}$ that satisfies the following condition:

$$
\begin{align*}
a_{ij}=\begin{cases}
255 & ((i - x)^2 + (j - y)^2 \leq r^2) \\
0 & (\text{otherwise})
\end{cases}
.\end{align*}
$$

- $h, w, x, y, r$: `int`
- $10^2 \leq w, h \leq 10^3$
- $0 \leq x < h,~0 \leq y < w$
- $0 \leq r < \text{min}(h, w)$
- Return: `np.ndarray`
  - `shape`: `(h, w)`
  - `dtype`: `np.uint8`

<details><summary>tips</summary>

- [`np.meshgrid`](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html)

</details>

In [ ]:
def solution(h, w, x, y, r):
    # TODO
    ...


# Use the following codes to debug your implementation.
# import matplotlib.pyplot as plt
# plt.imshow(solution(200, 200, 40, 50, 30), cmap='gray', vmax=255)

test_func(solution, "03_01")
# show_solution("03_01")  # Uncomment it to see the solution.

Q3.2.

Some positive integers $h, w, x, y, r, s, t, u, v$ are given.
Continuing on from the previous question, we want to draw in addition a rectangle $L=\{(i, j)~|~s \leq i \leq t \land u \leq j \leq v\}$.
However, the overlapping region of the circle $C=\{(i, j)~|~(i-x)^2+(j-y)^2 \leq r^2\}$ and the rectangle, $D=C\cap L$, should be represented in a different color.
To fulfill these specifications, write code to create a matrix $A\in\mathbb{R}^{h \times w}$ that satisfies the following conditions:

$$
\begin{align*}
a_{ij}=\begin{cases}
255 & ((i, j) \in D ) \\
127 & ((i, j) \in C \setminus D ) \\
127 & ((i, j) \in L \setminus D ) \\
0 & (\text{otherwise})
\end{cases}
.\end{align*}
$$

- $h, w, x, y, r, s, t, u, v$: `int`
- $10^2 \leq w, h \leq 10^3$
- $0 \leq x < h,~0 \leq y < w$
- $0 \leq r < \text{min}(h, w)$
- $0 \leq s < t < h,~0 \leq u < v < w$
- Return: `np.ndarray`
  - `shape`: `(h, w)`
  - `dtype`: `np.uint8`

<details><summary>tips</summary>

- [`Logical operations`](https://numpy.org/doc/stable/reference/routines.logic.html#logical-operations)

</details>

In [ ]:
def solution(h, w, x, y, r, s, t, u, v):
    # TODO
    ...


# Use the following codes to debug your implementation.
# import matplotlib.pyplot as plt
# plt.imshow(solution(200, 200, 60, 50, 30, 10, 100, 30, 50), cmap='gray', vmax=255)

test_func(solution, "03_02")
# show_solution("03_02")  # Uncomment it to see the solution.

Q3.3.

In the previous section, we introduced the one-hot vectorization operation.
Let's implement the inverse transformation of this operation.
A matrix $A\in\mathbb{R}^{n\times 10}$ is given.
The $i$-th row $a_i=[a_{i0},a_{i1},~\ldots,~a_{i9}]$ of $A$ is a unit vector where one element is `1` and the rest are `0`.
Write code to output an array containing $k$ for each $i$, where $a_{ik}=1$.

- $A$: `np.ndarray`
  - `shape`: `(n, 10)`
  - `dtype`: `np.uint8`
- $1 \leq n \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.uint8`
- Sample
  - `array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])`->`array([2, 3])`

<details><summary>tips</summary>

- [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html)
- [`np.argmin`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "03_03")
# show_solution("03_03")  # Uncomment it to see the solution.

Q3.4.

A matrix $A=(a_{ij})_{0\leq i < m, 0 \leq j < n}$ and a positive integer $k$ is given.
Write code to retrieve the indices $(i, j)$ of the top $k$ highest values in $A$, sorted in descending order.

- $A$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.float64`
- $k$: `int`
- $1 \leq m, n \leq 10^{2}$
- $1 \leq k \leq \text{min}(mn, 10^2)$
- Return: `np.ndarray`
  - `shape`: `(k, 2)`
  - `dtype`: `np.int64`
- Sample
  - `array([[3., 7., 4.], [6., 5., 8]])`->`array([[1, 2],[0, 1]])`

<details><summary>tips</summary>

- [`np.argsort`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html)
- [`np.unravel_index`](https://numpy.org/doc/stable/reference/generated/numpy.unravel_index.html)

</details>

In [ ]:
def solution(arr, k):
    # TODO
    ...


test_func(solution, "03_04")
# show_solution("03_04")  # Uncomment it to see the solution.

Q3.5.

Matrices $A, B\in\mathbb{R}^{m\times n}$ are given.
Write code to create a matrix $C=(\text{max}(a_{ij}, b_{ij}))_{0\leq i < m, 0\leq j < n}$, which contains the larger element from $A$ and $B$ for each corresponding position.

- $A, B$: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64`
- $1 \leq m, n \leq 10^{2}$
- Return: `np.ndarray`
  - `shape`: `(m, n)`
  - `dtype`: `np.int64`

<details><summary>tips</summary>

- [`np.where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html)
- [`np.maximum`](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html)

</details>

In [ ]:
def solution(arr_a, arr_b):
    # TODO
    ...


test_func(solution, "03_05")
# show_solution("03_05")  # Uncomment it to see the solution.

## 4. Math and statistics

Next, we will learn some important mathematical and statistical functions.
NumPy provides elementary functions such as `np.sin` and `np.exp`, as well as statistical functions like `np.min` and `np.max` (for obtaining the minimum and maximum values) and `np.mean` and `np.var` (for calculating the mean and variance).
These functions will appear repeatedly in later chapters, so let's master how to use them.

Q4.1.

Over $t$ time steps, the XYZ coordinates of $n$ drones around the origin are given as a 3-dimensional tensor $A\in\mathbb{R}^{t\times n \times 3}$.
We want to obtain the spherical coordinates around the origin of each drone, and output the results in a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$ of the same size.
The transformation from XYZ coordinates $(x,y,z)$ to spherical coordinates $(r,\theta,\phi)$ is calculated using [the following equations](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Coordinate_system_conversions):

$$
\begin{align*}
r &= \sqrt{x^2 + y^2 + z^2} \\
\theta &= \mathrm{Arccos}{\frac{z}{\sqrt{x^2 + y^2 + z^2}}} \\
\phi &= \mathrm{sgn}(y)\mathrm{Arccos}{\frac{x}{\sqrt{x^2 + y^2}}}
.\end{align*}
$$

Write code to perform this computation.

- $A$: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`
- $10^2 \leq t \leq 10^3$
- $1 \leq n \leq 10^2$
- $0.1 \leq \sqrt{\sum_{k=0}^{1} a_{ijk}^2} \leq \sqrt{\sum_{k=0}^{2} a_{ijk}^2} \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`

<details><summary>tips</summary>

- [Mathematical functions](https://numpy.org/doc/stable/reference/routines.math.html)
- [`np.arccos`](https://numpy.org/doc/stable/reference/generated/numpy.arccos.html)
- [`np.sign`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html)
- [`np.cumsum`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "04_01")
# show_solution("04_01")  # Uncomment it to see the solution.

Q4.2.

In the Olympic Games, the trimmed mean method, which excludes the highest and lowest scores, is often used for scoring.
Now, data for $n$ athletes scored by 10 judges is provided as a matrix $A\in\mathbb{R}^{n\times 10}$.
For each athlete, the final score is determined by the sum of the 8 scores excluding the highest and lowest scores.
Write code to output an array containing the final scores for the $n$ athletes.

- $A$: `np.ndarray`
  - `shape`: `(n, 10)`
  - `dtype`: `np.int64`
- $1 \leq n \leq 10^{3}$
- $0 \leq a_{ij} \leq 10^2$
- Return: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int64`
- Sample
  - `array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])`->`array([44])`
  - `array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])`->`array([5])`

<details><summary>tips</summary>

- [`np.min`](https://numpy.org/doc/stable/reference/generated/numpy.min.html)
- [`np.max`](https://numpy.org/doc/stable/reference/generated/numpy.max.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "04_02")
# show_solution("04_02")  # Uncomment it to see the solution.

Q4.3.

A positive integer $m$ and an array $A\in\mathbb{R}^{t}$ containing sensor data over $t$ time steps are given.
To minimize the impact of noise, we want to create a new array $B=(b_{i})$ by applying the smoothing process as shown below:

$$
\begin{align*}
b_{i} &= \frac{1}{m}\sum\limits_{u=0}^{m-1} a_{i-u}\\
a_{i} &= \begin{cases}
  a_{i} & (i \geq 0) \\
  a_{0} & (\text{otherwise})
\end{cases}
.\end{align*}
$$

Write code to output the smoothed data array.

- $m$: `int`
- $A$: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- $1 \leq m \leq t \leq 10^{3}$
- $0 \leq |a_{ij}| \leq 10$
- Return: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- Sample
  - `3, np.array([1., 1., 1., 4., 7.])`->`array([1., 1., 1., 2., 4.])`

<details><summary>tips</summary>

- [`np.convolve`](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html)

</details>

In [ ]:
def solution(m, arr):
    # TODO
    ...


test_func(solution, "04_03")
# show_solution("04_03")  # Uncomment it to see the solution.

Q4.4.

Matrices $Y=(y_{ij}), D=(d_{ij}) \in \mathbb{R}^{t\times p}$, containing predicted data and true data for $p$ types of sensors over $t$ time steps, are given.
We want to calculate the normalized root mean squared error (NRMSE) for each sensor.
The NRMSE for the $k$-th data (where $0\leq k < p$) is defined by the following equation:

$$
\begin{align*}
\mathrm{NRMSE}_k &:= \frac{\mathrm{RMSE}_k}{\sigma_k}\\
\mathrm{RMSE}_k &:= \sqrt{\mathrm{MSE}_k} \\
&=\sqrt{\frac{1}{t}\sum\limits_{u=0}^{t-1}(y_{uk}-d_{uk})^2}\\
\sigma_k &:= \sqrt{\mathrm{Var}[d_k]} \\
&=\sqrt{\frac{1}{t}\sum\limits_{u=0}^{t-1} (d_{uk} - \mu_k)^2} \\
\mu_k &:=\frac{1}{t}\sum\limits_{u=0}^{t-1} d_{uk}
.\end{align*}
$$

Write code to output an array of size $p$, containing the NRMSE for each sensor.

- $Y, D$: `np.ndarray`
  - `shape`: `(t, p)`
  - `dtype`: `np.float64`
- $10^2 \leq t \leq 10^{3}$
- $1 \leq p \leq 10^{2}$
- $0 \leq |d_{ij}|, |y_{ij}| \leq 10$
- Return: `np.ndarray`
  - `shape`: `(p,)`
  - `dtype`: `np.float64`

<details><summary>tips</summary>

- [`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
- [`np.average`](https://numpy.org/doc/stable/reference/generated/numpy.average.html)
- [`np.var`](https://numpy.org/doc/stable/reference/generated/numpy.var.html)
- [`np.std`](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
- [`np.sqrt`](https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html)

</details>

In [ ]:
def solution(y, d):
    # TODO
    ...


test_func(solution, "04_04")
# show_solution("04_04")  # Uncomment it to see the solution.

Q4.5.

An array $A$ consisting of $n$ single-digit integers (0 to 9) is given.
Our aim is to count the frequency of each digit $k$ and obtain the probability distribution $P(k)$.
Then, we will calculate the entropy $H_2(P)$ defined by the following formula:

$$
\begin{align*}
H_2(P):=-\sum\limits_{k=0}^9 P(k)\log_2(P(k))
.\end{align*}
$$

Write code to perform this computation.
Be careful to avoid division by zero.

- $A$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.int8`
- $1 \leq n \leq 10^{4}$
- Return: `float`
- Sample
  - `array([0, 1, 0, 1])`->`1.0`
  - `array([0, 1, 2, 3])`->`2.0`
  - `array([0, 0])`->`0.0`

<details><summary>tips</summary>

- [`np.bincount`](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html)
- [`np.log`](https://numpy.org/doc/stable/reference/generated/numpy.log.html)
- [`np.log2`](https://numpy.org/doc/stable/reference/generated/numpy.log2.html)
- [`np.nansum`](https://numpy.org/doc/stable/reference/generated/numpy.nansum.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "04_05")
# show_solution("04_05")  # Uncomment it to see the solution.

## 5. Linear algebra

Finally, let's learn how to use NumPy methods related to linear algebra.
These methods are among the most powerful components of NumPy and play a vital role in this material as well.
In this chapter, we will focus particularly on the least squares method, while also covering major matrix operations such as computing inverse matrices, performing singular value decomposition, and finding eigenvalues.

Q5.1.

A 3-dimensional tensor $A\in\mathbb{R}^{t\times n \times 3}$ contains the XYZ coordinates of $n$ drones around the origin over $t$ time steps.
A matrix $O\in\mathbb{R}^{t\times 3}$ contains the observer's position coordinates.
An array $\Theta \in\mathbb{R}^{t}$ contains the angle of the observer's forward direction around the Z-axis.

Consider a coordinate system X'Y'Z' where the observer's position is the origin and the observer's forward direction is the X' axis.
Write code to compute a 3-dimensional tensor $B\in\mathbb{R}^{t\times n \times 3}$ containing the drone coordinates at each time step in the X'Y'Z' coordinate system.
When the angle of the observer's forward direction is $\theta$, the coordinate transformation matrix around the Z-axis is given by:

$$
\begin{align*}
x_\text{new} = \begin{bmatrix}
\cos \theta & \sin \theta & 0 \\
-\sin\theta & \cos \theta & 0 \\
0           & 0           & 1
\end{bmatrix} x_\text{old}
.\end{align*}
$$

- $A$: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`
- $O$: `np.ndarray`
  - `shape`: `(t, 3)`
  - `dtype`: `np.float64`
- $\Theta$: `np.ndarray`
  - `shape`: `(t,)`
  - `dtype`: `np.float64`
- $1 \leq t \leq 10^3$
- $1 \leq n \leq 10^2$
- Return: `np.ndarray`
  - `shape`: `(t, n, 3)`
  - `dtype`: `np.float64`

<details><summary>tips</summary>

- [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)

</details>

In [ ]:
def solution(arr, obs, angle):
    # TODO
    ...


# Use the following codes to debug your implementation.
# arr = [[[1, 0, 0]]]
# obs = [[-1, 0, -1]]
# angle = [np.pi / 2]
# arr, obs, angle = map(np.array, [arr, obs, angle])
# solution(arr, obs, angle)  # [[[0., -2., 1]]] is expected

test_func(solution, "05_01")
# show_solution("05_01")  # Uncomment it to see the solution.

Q5.2.

A matrix $A\in\mathbb{R}^{n \times p}$ is given.
For each row vector $a_k$, we want to normalize it so that its L2 norm becomes 1, as follows:

$$
\begin{align*}
\|a_k\|&=\sqrt{\sum\limits_{j=0}^{p-1}a_{kj}^2} \\
&= 1
.\end{align*}
$$

Write code to perform this computation.

- $A$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $1 \leq n \leq 10^{3}$
- $1 \leq p \leq 10^{3}$
- Return: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- Relative error: `1e-9`
- Sample
  - `2 * np.eye(3)`->`array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])`

<details><summary>tips</summary>

- [`np.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "05_02")
# show_solution("05_02")  # Uncomment it to see the solution.

Q5.3.

The least squares problem is an important problem frequently encountered in scientific computing.
It is formulated as an optimization problem to find $\beta\in\mathbb{R}^{p}$ that minimizes $\mathcal{L}(\beta)=\|X\beta-Y\|^2$, given a matrix $X\in\mathbb{R}^{n\times p}$ and an array $Y\in\mathbb{R}^{n}$.
NumPy provides the [`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) function to compute the least squares solution, which is recommended for practical use.
However, here we implement it without using this function to understand the underlying mechanism and practice NumPy.

In machine learning and statistics, $X$ is also called the design matrix, which contains $n$ data points with $p$ explanatory variables each.
Consider the case where $n>p$, meaning a sufficient number of data points is available.
Under this condition, the least squares solution $\hat{\beta}$ that minimizes $\mathcal{L}(\beta)$ can be explicitly obtained by the following equation (detailed derivation is left to the reader):

$$
\begin{align*}
\hat{\beta} = (X^\top  X)^{-1} X^\top Y
.\end{align*}
$$

In addition, the residual sum of squares (RSS) under this optimal $\hat{\beta}$, denoted as $\mathrm{RSS}:=\mathcal{L}(\hat{\beta})$, is obtained by the following equation:

$$
\begin{align*}
\mathrm{RSS}
&= \|X\hat{\beta} - Y\|^2 \\
&= \|X(X^\top  X)^{-1}X^\top  Y - Y\|^2 \\
&= \|\left(I - X(X^\top  X)^{-1}X^\top \right)Y\|^2
.\end{align*}
$$

Based on these explanations, write code to compute $\mathrm{RSS}$ for the $X, Y$ given.
As previously mentioned, do not use `np.linalg.lstsq` for this problem.
The answers will be validated against the output of `np.linalg.lstsq`.

- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $Y$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

<details><summary>tips</summary>

- [`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)
- [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)

</details>

In [ ]:
def solution(x, y):
    # TODO DO NOT USE `np.linalg.lstsq`.
    ...


test_func(solution, "05_03")
# show_solution("05_03")  # Uncomment it to see the solution.

Q5.4.

Building on the previous question, let us deepen our understanding of the least squares problem.
Computing the least squares solution $\hat{\beta}$ involves the inverse matrix $(X^\top  X)^{-1}$.
As is well known, the inverse matrix exists only when $X^\top  X$ is full-rank, meaning it has rank $p$ under the condition $n \geq p$.
Moreover, the inverse matrix computation becomes unstable when the absolute values of eigenvalues are small.
Due to these numerical constraints and instabilities, methods based on [singular value decomposition (SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition) are often used instead of direct matrix inversion (indeed, `np.linalg.lstsq` internally uses LAPACK's `dgelsd()`, which employs SVD).
SVD decomposes the matrix $X$ into orthogonal matrices $U\in\mathbb{R}^{n\times p}, V\in\mathbb{R}^{p\times p}$ and a diagonal matrix $\Sigma=\mathrm{diag}(\sigma_0, \sigma_1,~\ldots,~\sigma_p)\in\mathbb{R}^{p\times p}$ containing *singular values* on its diagonal, as follows:

$$
\begin{align*}
X = U \Sigma V^\top
.\end{align*}
$$

The term $(X^\top X)^{-1}X^\top $ appearing in the expression for $\hat{\beta}$ is called the [pseudoinverse](https://en.wikipedia.org/wiki/Generalized_inverse) (denoted hereafter as $X^+$).
Using SVD, the pseudoinverse $X^+$ can be expressed as follows (the proof is omitted; readers are encouraged to derive it themselves):

$$
\begin{align*}
X^+=V\Sigma^+ U^\top
,\end{align*}
$$

where $\Sigma^+$ is a diagonal matrix constructed by taking the reciprocals of the non-zero diagonal elements (singular values).
For quantitative evaluation of the solution, the coefficient of determination $\mathrm{R}^2$ is often used.
$\mathrm{R}^2$ indicates how well the explanatory variables $X$ can explain the target variable $Y$.
$\mathrm{R}^2$ is obtained from the following formula, and values closer to 1 indicate better fit:

$$
\begin{align*}
\mathrm{R}^2&:=1-\frac{\mathrm{RSS}}{n\mathrm{Var}[Y]}\\
&=1-\frac{\mathrm{RSS}}{\sum\limits_{i}^{n} (y_i-\mathrm{E}[Y])^2}
.\end{align*}
$$

Here we aim to compute $\mathrm{R}^2$ without using the inverse matrix.
For simplicity, assume that $Y$ is normalized so that its mean $\mathrm{E}[Y]=\frac{1}{n}\sum\limits_{j=1}^{n}y_j$ equals zero.
Using the notation $\|Y\|^2=n\mathrm{Var}[Y]$ and substituting the formula for $\mathrm{RSS}$ from the previous question, we obtain:

$$
\begin{align*}
\mathrm{R}^2&:=1-\frac{\mathrm{RSS}}{\|Y\|^2}\\
&= 1-\frac{\|\left(I - X(X^\top  X)^{-1}X^\top \right)Y\|^2}{\|Y\|^2} \\
&= 1-\frac{\|\left(I - P_X\right)Y\|^2}{\|Y\|^2} \\
&= 1-\frac{Y^\top \left(I - P_X\right)^\top \left(I - P_X\right)Y}{\|Y\|^2} \\
&= 1-\frac{Y^\top \left(I - P_X - P_X^\top  + P_X^\top  P_X\right)Y}{\|Y\|^2} \\
&= \frac{Y^\top \left(P_X + P_X^\top  - P_X^\top  P_X\right)Y}{\|Y\|^2}
,\end{align*}
$$

where $P_X= X(X^\top  X)^{-1}X^\top $ is called the projection matrix, an important symmetric matrix satisfying $P_X^2=P_X$ and $P_X^\top =P_X$ (this can be verified by direct substitution; geometrically, it projects $Y$ onto the subspace spanned by $X$).
Combining this with the SVD of $X$ and $X^+$, we can transform $\mathrm{R}^2$ as follows:

$$
\begin{align*}
\mathrm{R}^2&=\frac{Y^\top  P_X Y}{\|Y\|^2}\\
&=\frac{Y^\top  X(X^\top  X)^{-1}X^\top  Y}{\|Y\|^2}\\
&=\frac{Y^\top  X X^+ Y}{\|Y\|^2}\\
&=\frac{Y^\top  U\Sigma V^\top  V \Sigma^+ U^\top  Y}{\|Y\|^2}\\
&=\frac{Y^\top  U U^\top  Y}{\|Y\|^2}\\
&=\frac{\|U^\top  Y\|^2}{\|Y\|^2}
.\end{align*}
$$

From the triangle inequality and orthogonality, we have $0 \leq \mathrm{R}^2\leq 1$ (since $P_X Y \perp (Y - P_X Y)$, we have $\|Y\|^2 = \|P_X Y\|^2+ \|Y - P_X Y\|^2$).

Now for the task.
Matrices $X\in\mathbb{R}^{n\times p}$ and $Y\in\mathbb{R}^{n}$ are given, where $Y$ is normalized such that $\sum_{j=1}^{n} y_j=0$.
Write code to compute the coefficient of determination $\mathrm{R}^2$ of $X$ with respect to $Y$.
As in the previous question, do not use `np.linalg.lstsq`; your answer will be validated using `np.linalg.lstsq`.

- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $Y$: `np.ndarray`
  - `shape`: `(n,)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

<details><summary>tips</summary>

- [`np.linalg.pinv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html)
- [`np.linalg.svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)

</details>

In [ ]:
def solution(x, y):
    # TODO DO NOT USE `np.linalg.lstsq` for your answer.
    ...


test_func(solution, "05_04")
# show_solution("05_04")  # Uncomment it to see the solution.

Q5.5.

A design matrix $X\in\mathbb{R}^{n\times p}$ is given.
Let $C=\mathrm{E}[(X-\mathrm{E}[X])(X-\mathrm{E}[X])^\top ]$ $\left(\mathrm{E}[X]:=\frac{1}{n}\sum\limits_{i=1}^{n-1}X_{ij}\right)$ be the covariance matrix, and let $(\lambda_i)_{0\leq i < p}$ be the absolute values of its eigenvalues.
The [effective dimension](https://arxiv.org/abs/0912.3832) $N_\text{eff}$ of $X$ is defined by the following equation:

$$
\begin{align*}
N_\text{eff} &:= \frac{\left(\sum\limits_{i=0}^{p-1}\lambda_i\right)^2}{\sum\limits_{i=0}^{p-1}\lambda_i^2} \\
&=\left(\sum\limits_{i=0}^{p-1}\tilde{\lambda}_i^2\right)^{-1}
,\end{align*}
$$

where $(\tilde{\lambda}_i)_{0\leq i < p}$ are the eigenvalues uniformly scaled so that their sum equals 1.

Write code to compute the effective dimension.

- $X$: `np.ndarray`
  - `shape`: `(n, p)`
  - `dtype`: `np.float64`
- $1 \leq p  \leq 10^2$
- $p \leq n \leq 10^{3}$
- Return: `float`

<details><summary>tips</summary>

- [`np.linalg.eig`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html)

</details>

In [ ]:
def solution(arr):
    # TODO
    ...


test_func(solution, "05_05")
# show_solution("05_05")  # Uncomment it to see the solution.

## Reference

[1] *NumPy - Learn*, NumPy team. https://numpy.org/learn/

[2] Abbott, L. F., Rajan, K., & Sompolinsky, H. (2010). Interactions between Intrinsic and Stimulus-Evoked Activity in Recurrent Neural Networks (No. arXiv:0912.3832). arXiv. https://doi.org/10.48550/arXiv.0912.3832